# Read in parameter text file ('par_values.txt')

In [1]:
f = open('par_values.txt')
par_dict = {}
for line in f:
    line = line.strip()
    if not line: continue

    if line.startswith('Parameters'):
        if line not in par_dict: par_dict[line] = []
        heading = line
        continue
    par_dict[heading].append(line)

f.close()

# Select appropriate parameters for blood type (adult vs. fetal) and field strength (1.5T vs. 3T)

In [2]:
import some_funcs as sf
#Fetch parameters for fetal blood at 1.5T.
model_pars = sf.fetch_model_pars(par_dict, blood_type='Fetal', field_strength='1.5')  

# Calculate intermediate coefficients, accounting for refocusing interval of T<sub>2</sub> acquisition

In [3]:
coeff_list = sf.calc_coeffs(model_pars, tau_180=0.016) #tau_180 is refocusing interval (in seconds)

# Calculate hematocrit (Hct) from oxygen-saturation (sO<sub>2</sub>) and T<sub>1</sub>:

In [4]:
Hct = sf.Hct_from_sO2_T1(coeff_list, sO2=0.85, T1=1.5)

Hct=0.41


# Calculate oxygen-saturation (sO<sub>2</sub>) from hematocrit (Hct) and T<sub>2</sub>: 
### NOTE: We are solving a quadratic, so there will be TWO solutions for sO<sub>2</sub>. Typically only one of these will be physical (i.e. 0&lt;sO<sub>2</sub>&lt;1)

In [5]:
sO2 = sf.sO2_from_Hct_T2(coeff_list, Hct=0.45, T2=0.1)

solution 1: sO2=0.32
solution 2: sO2=1.79


# Calculate oxygen saturation (sO<sub>2</sub>) and hematocrit (Hct) from T<sub>1</sub> and T<sub>2</sub>: 
### NOTE: We are solving a cubic, so there will be THREE solutions for (sO<sub>2</sub>, Hct).  In most cases, only one solution will be physical (i.e. 0&lt;sO<sub>2</sub>&lt;1 AND 0&lt;Hct&lt;1).  

In [6]:
Hct, sO2 = sf.Hct_sO2_from_T1_T2(coeff_list, T1=1.2, T2=0.1)

solution 1: sO2 = 1.54 + 0.20i, Hct = 1.09 + 0.18i
solution 2: sO2 = 1.54 - 0.20i, Hct = 1.09 - 0.18i
solution 3: sO2 = 0.34, Hct = 0.56


## For interpretation of cases where there is no physical solution or more than one physical solution, please refer to: Portnoy et al, <i>Magn Reson Med</i>, 2017, doi: 10.1002/mrm.26599.